In [1]:
import Foundation
import TensorFlow

In [2]:
let trainCSV = try String(contentsOfFile:"../data/train.csv", encoding: String.Encoding.utf8)
let testCSV = try String(contentsOfFile:"../data/test.csv", encoding: String.Encoding.utf8)

let trainRecords: [[Float]] = trainCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }
let testRecords: [[Float]] = testCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }

let numTrainRecords = trainRecords.count
let numTrainColumns = trainRecords[0].count
let numTestRecords = testRecords.count
let numTestColumns = testRecords[0].count

print(numTrainRecords, numTrainColumns, numTestRecords, numTestColumns)

let xTrain = trainRecords.map{ Array($0[0..<numTrainColumns-1]) }
let yTrain = trainRecords.map{ [$0[numTrainColumns-1]] }
let xTest = testRecords.map{ Array($0[0..<numTestColumns-1]) }
let yTest = testRecords.map{ [$0[numTestColumns-1]] }

print(xTrain.count, xTrain[0].count, yTrain.count, yTrain[0].count,
      xTest.count, xTest[0].count, yTest.count, yTest[0].count)

let xAllTrain = Array(xTrain.joined())
let yAllTrain = Array(yTrain.joined())
let xAllTest = Array(xTest.joined())
let yAllTest = Array(yTest.joined())

print(xAllTrain.count, yAllTrain.count, xAllTest.count, yAllTest.count)

405 14 101 14
405 13 405 1 101 13 101 1
5265 405 1313 101


In [3]:
let XTrain = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([numTrainRecords, numTrainColumns-1]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, 1]))
let XTest = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numTestRecords, numTestColumns-1]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, 1]))

print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

[405, 13] [405, 1] [101, 13] [101, 1]


In [4]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

In [5]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [6]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(xTrain.count) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [7]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(xTrain.count, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let logits = model(XTrain[batchStart..<batchEnd])
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(XTrain[batchStart..<batchEnd])
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 545.4098, MAE: 21.31943
Epoch 2: MSE: 476.44434, MAE: 19.698984
Epoch 3: MSE: 405.85806, MAE: 17.948347
Epoch 4: MSE: 328.4917, MAE: 15.852579
Epoch 5: MSE: 248.72086, MAE: 13.455591
Epoch 6: MSE: 175.22458, MAE: 10.922548
Epoch 7: MSE: 119.29696, MAE: 8.771885
Epoch 8: MSE: 81.9611, MAE: 7.1040225
Epoch 9: MSE: 60.094975, MAE: 5.879731
Epoch 10: MSE: 46.4849, MAE: 5.025765
Epoch 11: MSE: 37.470028, MAE: 4.456721
Epoch 12: MSE: 31.817493, MAE: 4.0055914
Epoch 13: MSE: 28.276432, MAE: 3.6853185
Epoch 14: MSE: 25.877867, MAE: 3.5092404
Epoch 15: MSE: 24.278948, MAE: 3.3287835
Epoch 16: MSE: 22.777208, MAE: 3.2087636
Epoch 17: MSE: 21.446352, MAE: 3.1102715
Epoch 18: MSE: 20.439192, MAE: 3.0103197
Epoch 19: MSE: 19.490509, MAE: 2.9020119
Epoch 20: MSE: 18.735216, MAE: 2.8760371
Epoch 21: MSE: 17.84153, MAE: 2.7689197
Epoch 22: MSE: 17.312727, MAE: 2.7176208
Epoch 23: MSE: 16.793966, MAE: 2.6954756
Epoch 24: MSE: 16.335466, MAE: 2.6194148
Epoch 25: MSE: 15.86725, MAE: 2.57424

Epoch 203: MSE: 4.878366, MAE: 1.4042605
Epoch 204: MSE: 4.7539563, MAE: 1.4358004
Epoch 205: MSE: 5.0501184, MAE: 1.4189283
Epoch 206: MSE: 4.992231, MAE: 1.4041361
Epoch 207: MSE: 4.8274317, MAE: 1.4024968
Epoch 208: MSE: 4.88775, MAE: 1.4105562
Epoch 209: MSE: 4.807926, MAE: 1.4079629
Epoch 210: MSE: 4.8202066, MAE: 1.3916783
Epoch 211: MSE: 4.793727, MAE: 1.3870043
Epoch 212: MSE: 4.648814, MAE: 1.374473
Epoch 213: MSE: 4.8756137, MAE: 1.386986
Epoch 214: MSE: 4.796791, MAE: 1.3813362
Epoch 215: MSE: 4.6839, MAE: 1.3748125
Epoch 216: MSE: 4.664207, MAE: 1.374256
Epoch 217: MSE: 4.6198535, MAE: 1.3662976
Epoch 218: MSE: 4.687263, MAE: 1.3618867
Epoch 219: MSE: 4.593243, MAE: 1.3622469
Epoch 220: MSE: 4.6470447, MAE: 1.3693023
Epoch 221: MSE: 4.6744757, MAE: 1.3547293
Epoch 222: MSE: 4.609277, MAE: 1.3592956
Epoch 223: MSE: 4.526564, MAE: 1.3485408
Epoch 224: MSE: 4.6689897, MAE: 1.3519263
Epoch 225: MSE: 4.457764, MAE: 1.3522359
Epoch 226: MSE: 4.5753145, MAE: 1.3448695
Epoch 227: M

Epoch 404: MSE: 2.5881286, MAE: 0.9550529
Epoch 405: MSE: 2.6011128, MAE: 0.9577672
Epoch 406: MSE: 2.5436842, MAE: 0.96025497
Epoch 407: MSE: 2.6398947, MAE: 0.9609563
Epoch 408: MSE: 2.5852323, MAE: 0.9521199
Epoch 409: MSE: 2.534275, MAE: 0.951603
Epoch 410: MSE: 2.5856986, MAE: 0.95750904
Epoch 411: MSE: 2.4998286, MAE: 0.9502793
Epoch 412: MSE: 2.6208074, MAE: 0.9443017
Epoch 413: MSE: 2.5560546, MAE: 0.9524181
Epoch 414: MSE: 2.5395637, MAE: 0.9479866
Epoch 415: MSE: 2.4709246, MAE: 0.93597317
Epoch 416: MSE: 2.5074232, MAE: 0.9314384
Epoch 417: MSE: 2.5197995, MAE: 0.93761355
Epoch 418: MSE: 2.5512264, MAE: 0.94585335
Epoch 419: MSE: 2.5458684, MAE: 0.935032
Epoch 420: MSE: 2.4604356, MAE: 0.94099563
Epoch 421: MSE: 2.4843047, MAE: 0.93805987
Epoch 422: MSE: 2.469322, MAE: 0.94427997
Epoch 423: MSE: 2.5015407, MAE: 0.93917465
Epoch 424: MSE: 2.4572797, MAE: 0.9273147
Epoch 425: MSE: 2.4602132, MAE: 0.9233844
Epoch 426: MSE: 2.4245634, MAE: 0.91762984
Epoch 427: MSE: 2.4300694, M